# Part1: Collecting Data from Skeleton Info

In [ ]:
import pandas as pd
import numpy as np
import os
import json
import csv
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Bidirectional, Embedding, LSTM
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras import backend as kb
from keras import initializers
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
from highcharts import Highchart

In [ ]:
def getJsonList(filePath):
    fileList = []
    for i,j,k in os.walk(filePath):
        fileList = k
    return fileList

def calcWeight(item):
    zeros = 0
    for i in item['pose_keypoints_2d']:
        if i == 0:
            zeros += 1
    for i in item['face_keypoints_2d']:
        if i == 0:
            zeros +=1
    return zeros

def getJsonForOneFrame(fileName, filePath):
    l = getJsonList(filePath)
    with open(fileName,'r') as f:
        temp = json.loads(f.read())
        allPeople = temp['people']
    
    if len(allPeople) == 0:
        empty = []
        for i in range(190):
            empty.append(0)
        return empty
    target = allPeople[0]
    for dic in allPeople:
        if calcWeight(dic) < calcWeight(target):
            target = dic
    matrix = []
    if(len(target['pose_keypoints_2d']) == 0):
        for i in range(50):
            matrix.append(0)
    else:
        for i in range(75):
            if (i+1) % 3 == 0:
                pass
            else:
                matrix.append(target['pose_keypoints_2d'][i])
                
    if(len(target['face_keypoints_2d']) == 0):
        for i in range(140):
            matrix.append(0)
    else:
        for i in range(210):
            if (i+1) % 3 == 0:
                pass
            else:
                matrix.append(target['face_keypoints_2d'][i])
    return matrix

In [ ]:
def getLable(dataCat, name, init):
    #print(name, init)
    dataCat = pd.read_csv('data.csv')
    for index, row in dataCat.iterrows():
        if row['name'] == name:
            mark = row['mark']
            mark = mark.split('-')

            if len(mark) % 2 != 0 or len(mark) == 0:
                print("ERROR")
                return None
            flag = 0
            for i in range(30):
                slow = 0
                fast = 1
                while(fast < len(mark)):
                    small = mark[slow]
                    big = mark[fast]
                    if int(small) <= int(init) + i <= int(big):
                        flag += 1
                        break
                    else:
                        slow += 2
                        fast += 2
            #print(flag)
    if flag >= 25:
        return True
    else:
        return False

def writeData(path, pathType,fullDfList):
    fileList = getJsonList(path)
    fileList = set(fileList)
    print(len(fileList))
    dataCat = pd.read_csv('data.csv')
    for index, row in dataCat.iterrows():
        temp = int(row['name'][:-1])
        if(row["type"] != pathType):
            pass
        else:
            startName = row["name"]+"_"+'0'.zfill(12)+"_keypoints.json"
            endName = row["name"]+"_"+'30'.zfill(12)+"_keypoints.json"
            if endName and endName not in fileList:
                print('Video ',row["name"]," is not long enough")
            else:
                initFrame = 0
                while(row["name"]+"_"+str(initFrame + 29).zfill(12)+"_keypoints.json" in fileList):
                    matrix = []
                    for i in range(30):
                        file = row["name"]+"_"+str(initFrame + i).zfill(12)+"_keypoints.json"
                        matrixLine = getJsonForOneFrame(path+'/'+file,path)
                        matrix.append(matrixLine)
                    if row['type'] == 'shortyes':
                        label = True
                    elif row['type'] == 'shortno':
                        label = False
                    else:
                        label = getLable(dataCat,row['name'], initFrame)
                    data = [[label, matrix]]
                    print("Data:",label, row['name'], initFrame,'matrix:',len(matrix),len(matrix[0]))
                    df = pd.DataFrame(data,columns=['lable','matrix'])
                    fullDfList.append(df)
                    initFrame += 30
    return fullDfList

In [ ]:
fullDfList = []
fullDfList = writeData('output/long', 'long',fullDfList)
print(len(fullDfList))
fullDfList = writeData('output/shortno', 'shortno',fullDfList)
print(len(fullDfList))
fullDfList = writeData('output/shortyes', 'shortyes',fullDfList)
print(len(fullDfList))
df = pd.concat(fullDfList, axis=0 )
df.reset_index(drop=True, inplace=True)
print(df)
df.to_json("fulldata.json")

In [ ]:
fulldf = pd.read_json("fulldata.json")


s55 = []
s55 = writeData('55s', 'shortyes',s55)
print(s55)
df = pd.concat(s55, axis=0 )
df.reset_index(drop=True, inplace=True)
#print(df)
df.to_json("s55.json")
df55 = pd.read_json("s55.json")
print(df55)

n331 = []
n331 = writeData('331n', 'shortno',n331)
df = pd.concat(n331, axis=0 )
df.reset_index(drop=True, inplace=True)
#print(df)
df.to_json("n331.json")
df331 = pd.read_json("n331.json")
print(df331)

df = pd.concat([fulldf, df331, df55], axis=0 )
df.reset_index(drop=True, inplace=True)
print(df)
df.to_json("dataset.json")
fulldf = pd.read_json("dataset.json")
print(fulldf)

In [ ]:
fulldf = pd.read_json("dataset.json")
print(fulldf)
#cutdf = fulldf.sample(frac=0.1, replace=False)
cutdf = fulldf
print(cutdf)
x = cutdf['matrix']
y = cutdf['lable']
x_train, x_test, y_train,y_test = train_test_split(x, y, test_size=0.25)

x_train.reset_index(drop=True, inplace=True)
x_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
print('1')

In [ ]:
def cleanX(raw):
    length = len(raw)
    temp = np.zeros((length, 30,190))
    for i in range(len(raw)):
        for j in range(30):
            for k in range(190):
                temp[i][j][k] = raw[i][j][k]
        print("progress:{0}%".format(round((i + 1) * 100 / len(raw))), end="\r")
    return temp
x_test = cleanX(x_test)
x_train = cleanX(x_train)
#print(temp)
def cleanY(raw):
    res = []
    for i in range(len(raw)):
        if raw[i] == True:
            res.append(1)
        else:
            res.append(0)
    res = np.array(res)
    return res
y_train = cleanY(y_train)
y_test = cleanY(y_test)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
x_train = np.load("x_train.npy")
x_test = np.load("x_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")